In [1]:
# Description: This program is inspired by Youtube channel Computer Science at https://www.youtube.com/watch?v=bvDkel5whUY
#              It extracts live > 10Y historical data from Yahoo Finance and produces an optimized stock portfolio from S&P 500.

In [2]:
portfolio_val = int(input("How much do you want to invest in? "))

How much do you want to invest in? 100000


In [ ]:
pip install yfinance

In [4]:
import pandas_datareader as web
import numpy as np 
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

In [5]:
# Get Data from NASDAQ
print("\nDownloading 10Y Past Performance of S&P500 constituents from Yahoo Finance...\n")
start = datetime.datetime(2010,1,1)
end = datetime.datetime.now()

url="https://pkgstore.datahub.io/core/s-and-p-500-companies/constituents_csv/data/e613177765e570e43c2a1e8330bf73bf/constituents_csv.csv"
s = requests.get(url).content
companies = pd.read_csv(io.StringIO(s.decode('utf-8')))

Stocks = companies['Symbol'].tolist()

Stocks = [stock.replace('.', '-') for stock in Stocks]

df = yf.download(Stocks,start,end)["Adj Close"]

df = df.dropna(axis=1, how='all')



[*********************100%***********************]  505 of 505 completed


In [6]:
assets = df.columns

In [ ]:
pip install PyPortfolioOpt

In [8]:
# Optimize the Portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
# Calculate the expected annualised returns and the annualized sample covariance matrix of the daily asset returns
exp_r = expected_returns.mean_historical_return(df)
sam_c = risk_models.sample_cov(df)

In [10]:
# Optimize for miximal Sharpe ratio
ef = EfficientFrontier(exp_r, sam_c)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

OrderedDict([('A', 0.0), ('AAL', 0.0), ('AAP', 0.0), ('AAPL', 0.0), ('ABBV', 0.0), ('ABC', 0.0), ('ABMD', 0.0), ('ABT', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADM', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AEE', 0.0), ('AEP', 0.0), ('AES', 0.0), ('AFL', 0.0), ('AIG', 0.0), ('AIZ', 0.0), ('AJG', 0.0), ('AKAM', 0.0), ('ALB', 0.0), ('ALGN', 0.0), ('ALK', 0.0), ('ALL', 0.0), ('ALLE', 0.0), ('ALXN', 0.0), ('AMAT', 0.0), ('AMCR', 0.0), ('AMD', 0.0), ('AME', 0.0), ('AMGN', 0.0), ('AMP', 0.0), ('AMT', 0.0), ('AMZN', 0.0), ('ANET', 0.0), ('ANSS', 0.0), ('ANTM', 0.0), ('AON', 0.0), ('AOS', 0.0), ('APA', 0.0), ('APD', 0.0), ('APH', 0.0), ('APTV', 0.0), ('ARE', 0.0), ('ATO', 0.0), ('ATVI', 0.0), ('AVB', 0.0), ('AVGO', 0.0), ('AVY', 0.0), ('AWK', 0.0), ('AXP', 0.0), ('AZO', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BAX', 0.0), ('BBY', 0.0), ('BDX', 0.0), ('BEN', 0.0), ('BF-B', 0.0), ('BIIB', 0.0), ('BIO', 0.0), ('BK', 0.0), ('BKNG', 0.0), ('BKR', 0.0), ('BLK', 0.0), ('BLL', 0.0), ('BMY', 0.0), ('B

(0.9952381014789006, 0.27272345474708287, 3.5759230990356614)

In [ ]:
pip install pulp

In [12]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, portfolio_val)
allocation, leftover = da.lp_portfolio()
#print('Discrete allocation: ', allocation)
#print('Remaining Cash: $', leftover)

In [13]:
# Funcation that gets company name from its ticker
def get_name(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

In [14]:
# Get company names
company_list = []
for symbol in allocation:
  company_list.append(get_name(symbol))

In [15]:
# Get discrete allocation Values
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [16]:
# Create dataframe for portfolio
portfolio_df = pd.DataFrame(columns = ['Company name', 'Ticker', 'No. of shares to hold'])

In [17]:
portfolio_df['Company name'] = company_list
portfolio_df['Ticker'] = allocation
portfolio_df['No. of shares to hold'] = discrete_allocation_list

In [18]:
# Show the portfolio
print("********* Recommended Portfolio for USD$",portfolio_val," *********\n")
print(portfolio_df)
print("\n")
print('Remaining Cash: $', leftover)

********* Recommended Portfolio for USD$ 100000  *********

                      Company name Ticker  No. of shares to hold
0       Carrier Global Corporation   CARR                    868
1     Charter Communications, Inc.   CHTR                      6
2               The Clorox Company    CLX                     84
3     Costco Wholesale Corporation   COST                      3
4       Dollar General Corporation     DG                      8
5             Domino's Pizza, Inc.    DPZ                     67
6                    Netflix, Inc.   NFLX                     12
7                 Pool Corporation   POOL                      4
8  Regeneron Pharmaceuticals, Inc.   REGN                      4
9           WEC Energy Group, Inc.    WEC                     15


Remaining Cash: $ 14.509323120117188
